In [ ]:
import pandas as pd

from dotenv import load_dotenv
load_dotenv()
import os

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

##GVOL
from gvol import GVol

##USE .ENV FILE
GVOL_API_KEY = os.getenv('GVOL_API_KEY')
##OR INSERT MANUALLY YOU KEYS
#GVOL_API_KEY = 'GVOL_API_KEY'

gvol_client = GVol(header='x-oracle',gvol_api_key=GVOL_API_KEY)

In [ ]:
##SELECT COIN BTC/ETH
symbol = 'BTC'

# CUMULATIVE NET VOLUMES

This endpoint returns the cumulative net volumes of trades for the last "n" days selected.

For calculating the "net" volume (aka the volume traded with the sign of the initiator) we use our proprietary algorithm composed from several heuristics which use the orderbook previous of the trade at millisecond granularity. You can read more about this here Gvol Direction.

The endpoint is completed with some useful filters, such as:

    - tradeType = ALL/block/onScreen
    
    - showActiveExpirations:
        true = endpoint returns only trades for active expirations
        false = endpoint returns all the trades even for expired expirations.

In [ ]:
##Params
active_exp = False 
days = 10
type_of_trade = 'ALL' 

In [ ]:
data = gvol_client.options_cumulative_net_volumes(symbol= symbol , exchange= 'deribit', showActiveExpirations= active_exp, days=days, tradeType=type_of_trade)
df = pd.json_normalize(data['genericNetVolumeGvolDirection'])
df['date'] = pd.to_datetime(df['date'], unit='ms')
df.set_index('date', inplace=True)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

zmax = max(abs(df['cumulative']))

fig.add_heatmap(z=df.cumulative,
        x=df.index,
        y=df.strike,
        zmin = -zmax,
        zmax = +zmax,
        colorscale='RdBu'
)

price = df.reset_index().groupby('date').mean()['indexPrice']
fig.add_trace(go.Scatter(x = price.index, y = price.values, name = 'Index', line= dict(color='grey', dash='dot')))
if symbol == 'BTC':
    fig.update_yaxes(range=['12500','34500'], secondary_y=True)
    fig.update_yaxes(range=['12500','34500'], secondary_y=False)
else:
    fig.update_yaxes(range=['850','2400'], secondary_y=True)
    fig.update_yaxes(range=['850','2400'], secondary_y=False)

fig.add_annotation(text="AMBERDATA",opacity=0.5,
                  xref="paper", yref="paper",
                  x=0.16, y=0.1, showarrow=False)

fig.update_yaxes(title='strike', secondary_y=False)

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.update_layout(title=str(symbol)+' '+str(type_of_trade).upper()+ ' Cumulative Net Volumes',
           )

#fig.write_image('onscreen.svg', engine='kaleido', width=900, height=500)

fig.show()

# CUMULATIVE NET POSITIONING

This endpoint returns the cumulative net positioning of traders for the period from the dateStart parameter. It means that positioning is assumed "zero" at the dateStart.

This endpoint starts from 7th November 2022.

This endpoint could be seen as the other side of the gamma exposure of dealers (Gvol Gex).

In [ ]:
##Params
start_date = '2023-1-1'

In [ ]:
data = gvol_client.options_cumulative_net_positioning(symbol= symbol , exchange= 'deribit', dateStart=start_date)
df = pd.json_normalize(data['genericNetPositioningGvolDirection'])
df['date'] = pd.to_datetime(df['date'], unit='ms')
df.set_index('date', inplace=True)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

zmax = max(abs(df['netInv']))

fig.add_heatmap(z=df.netInv,
        x=df.index,
        y=df.strike,
        zmin = -zmax,
        zmax = +zmax,
        colorscale='RdBu'
)

price = df.reset_index().groupby('date').mean()['indexPrice']
fig.add_trace(go.Scatter(x = price.index, y = price.values, name = 'Index', line= dict(color='grey', dash='dot')))
if symbol == 'BTC':
    fig.update_yaxes(range=['12500','34500'], secondary_y=True)
    fig.update_yaxes(range=['12500','34500'], secondary_y=False)
else:
    fig.update_yaxes(range=['850','2400'], secondary_y=True)
    fig.update_yaxes(range=['850','2400'], secondary_y=False)

fig.add_annotation(text="AMBERDATA",opacity=0.5,
                  xref="paper", yref="paper",
                  x=0.16, y=0.1, showarrow=False)

fig.update_yaxes(title='strike', secondary_y=False)

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.update_layout(title=str(symbol)+' Cumulative Net Positioning',
           )

#fig.write_image('onscreen.svg', engine='kaleido', width=900, height=500)

fig.show()